In [11]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

import sparse_dot_topn as sdt
from scipy.sparse import csr_matrix

In [2]:
#import the dataSets
credit_df = pd.read_csv('/home/dans/Downloads/pyp/archive/credits.csv')
movies_df = pd.read_csv('/home/dans/Downloads/pyp/archive/movies_md.csv')
keyword = pd.read_csv('/home/dans/Downloads/pyp/archive/keywords.csv')

keyword


/tmp/ipykernel_6326/1405957569.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('/home/dans/Downloads/pyp/archive/movies_md.csv')


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [3]:
#merge the credits and the movies dataset and keywords into a variable named "movies_data"

credit_df['id'] = credit_df['id'].astype('string')

movies = movies_df.merge(credit_df, on='id')

keyword['id'] = keyword['id'].astype('string')

movies_data = movies.merge(keyword, on="id")

movies_data


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]"
46624,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46625,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[]
46626,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,[],Released,NaN,Sata

In [4]:
# Select specific columns from the dataset that i will be comparing
selected_columns = ['genres', 'id','adult','title', 'adult', 'original_language', 'keywords', 'cast' ]


In [5]:
#replacing null values with null strings
for column in selected_columns:
    movies_data[column] = movies_data[column].fillna('');


In [6]:
#select important columns
columns = movies_df['genres']+""+movies_data['cast']+""+movies_data['crew']+""+movies_data['adult']+""+movies_data['keywords']+""+movies_data['original_language']


In [13]:
# Convert the selected columns data into integers, which can be compared
columns = columns.astype("unicode")


vectorizer = TfidfVectorizer()

vectorizedColumns = vectorizer.fit_transform(columns).astype('float16')

svd = TruncatedSVD(n_components=10)


#sparse
def cossim_top(A, B, ntop, lower_bound):
    A = A.tocsr()
    B = B.tocsr()

    C = sdt.dot_topn(A, B.T, ntop, lower_bound)

    return C



ntop = 10
similarity_matrix = cossim_top(vectorizedColumns, vectorizedColumns, ntop, lower_bound=0.1)


vectorizedColumns



ImportError: cannot import name 'sparse_dot_topn' from 'sparse_dot_topn' (/home/dans/.local/lib/python3.10/site-packages/sparse_dot_topn/__init__.py)

In [ ]:
# find the Cosine_ similarity Score of the vectorized columns

#similarity = cosine_similarity(reducedColumns)

#similarity



NameError: name 'similarity' is not defined

In [ ]:
# Getting user Input 
userChoice = input('Enter Movie Name')


In [ ]:
# Generate all a list of all the movies in the dataset

moviesList = [movies_data['title'], movies_data['id']]



In [ ]:
# Compare user input to movieList and obtain the movie title from the list which is most similar (synatax) to the users input

getCloseMatche = difflib.get_close_matches(userChoice, moviesList[0])

# The get_close_matches functions returns an array, so we select the first item in the array
# which in theory should be closest match
matchedMovie = getCloseMatche[0]

matchedMovie

IndexError: list index out of range

In [ ]:
# Find the id of the matched movie from the dataset 'movies_data'

matchedMovieId = movies_data[movies_data.title == matchedMovie]['id'].values[0]

matchedMovieId

'19995'

In [ ]:
# Find the Cosine_Similarity Score for all the movies in the dataset

similarity_score = list(enumerate(similarity))

TypeError: 'str' object cannot be interpreted as an integer